<a href="https://colab.research.google.com/github/noumantechie/langchain/blob/main/retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU pypdf langchain_community

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load the PDF file
pdf_path = "/content/General_Instructions_candiadtes.pdf"  # Ensure the correct file path
loader = PyPDFLoader(pdf_path)





In [6]:
# Now use load_and_split()
pages = loader.load_and_split()

In [9]:
len(pages)

5

In [10]:
pages[1].page_content

'NATIONAL TEXTILE UNIVERSITY \n \nviii. The candidates who fail to furnish the requisite documents within stipulated time or \nfurnish incorrect/incomplete information, their candidature shall be liable to rejection. \nTherefore, they are advised to get their documents ready for submission. \nix. The candidates called for interview must bring their original documents etc. on the day \nof their interview. \nx. Eligibility of the candidates in all respects shall be reckoned up to the closing date. \nxi. The candidates are advised to apply online at the earliest without waiting for the closing \ndate. \n \nB. EDUCATIONAL QUALIFICATIONS AND EXPERIENCE \n \n1. The prescribed educational qualifications and valid registration with PEC , where \napplicable/required must have been acquired on or  before the closing date for submission of  \napplications.  \n \n2. Applications of candidates, whose result is not officially announced by the Controller of  \nExamination of a Board/ University on or